# Fama-French 3-factor model

In [91]:
%use kandy, 
%useLatestDescriptors
%use kandy
%use dataframe
%use lets-plot
import org.apache.commons.math3.stat.regression.OLSMultipleLinearRegression
import java.io.File
import org.jetbrains.letsPlot.geom.geomPoint
import org.jetbrains.letsPlot.ggplot
import org.jetbrains.letsPlot.label.ggtitle
import org.jetbrains.letsPlot.scale.scaleFillGradient
import org.jetbrains.letsPlot.geom.geomTile
import org.jetbrains.letsPlot.geom.geomHistogram
import org.jetbrains.letsPlot.geom.geomLine
import org.jetbrains.letsPlot.export.ggsave
import java.time.LocalDate
import java.time.format.DateTimeFormatter

### Import CSV data and setup StockData class

In [92]:
data class StockData(
    val date: String,
    val excess_return: Double,
    val mkt_rf: Double,
    val smb: Double,
    val hml: Double
)

// Read the CSV file
val data = File("../resources/CourseWork_Data_24-25.csv").readLines().drop(1)
    .map { line ->
        val parts = line.split(",")
        StockData(
            date = parts[0].replace("m", ""),
            excess_return = parts[5].toDouble(),
            mkt_rf = parts[10].toDouble(),
            smb = parts[11].toDouble(),
            hml = parts[12].toDouble()
        )
    }

### Prepare for and perform regression

In [93]:
// Prepare data for regression
val y = data.map { it.excess_return }.toDoubleArray()
val x = Array(data.size) { i ->
    doubleArrayOf(data[i].mkt_rf, data[i].smb, data[i].hml)
}

In [94]:
// Perform regression
val regression = OLSMultipleLinearRegression()
regression.newSampleData(y, x)

// Get coefficients
val coefficients = regression.estimateRegressionParameters()
val intercept = coefficients[0]
val betas = coefficients.sliceArray(1..coefficients.lastIndex)

// Calculate predicted excess returns
val predictedReturns = x.map { observation ->
    intercept + observation.zip(betas).sumOf { (x, beta) -> x * beta }
}

println("Intercept (Alpha): ${coefficients[0]}")
println("Market Beta: ${coefficients[1]}")
println("SMB coefficient: ${coefficients[2]}")
println("HML coefficient: ${coefficients[3]}")

// Calculate R-squared
val rSquared = regression.calculateRSquared()
println("R-squared: $rSquared")

Intercept (Alpha): 0.537858975867777
Market Beta: 1.1298036723246065
SMB coefficient: 0.15716569032486077
HML coefficient: -0.5355627459463376
R-squared: 0.8196864048503468


### Actual vs Predicted Excess Returns Scatter Plot:
This graph will help visualize how well the model fits the data.


In [95]:
val plotData = mapOf(
    "Actual" to y,
    "Predicted" to predictedReturns
)

val plot = ggplot(plotData) +
        geomPoint(color = "blue", alpha = 0.5) {
            x = "Actual"
            y = "Predicted"
        } +
        ggtitle("Actual vs Predicted Excess Returns")

plot.show()

### Residuals Plot: 
This can help identify any patterns in the residuals, which could indicate issues with the model.

In [96]:
val residuals = y.zip(predictedReturns).map { it.first - it.second }

val residualPlotData = mapOf(
    "Predicted" to predictedReturns.toList(),
    "Residuals" to residuals
)

val residualPlot = ggplot(residualPlotData) +
        geomPoint(color = "red", alpha = 0.5) {
            x = "Predicted"
            y = "Residuals"
        } +
        ggtitle("Residuals vs Predicted Values")

residualPlot.show()


### Factor Returns Over Time:
This can show how each factor (Market, SMB, HML) has performed over time.

In [97]:
import java.time.YearMonth
import java.time.ZoneOffset

val dateFormatter = DateTimeFormatter.ofPattern("yyyyMM")

val timeSeriesData = mapOf(
    "Date" to data.map { it.date },
    "Market" to data.map { it.mkt_rf },
    "SMB" to data.map { it.smb },
    "HML" to data.map { it.hml },
)

val factorReturnsPlot = ggplot(timeSeriesData) +
        geomLine { x = "Date"; y = "Market" } +
        geomLine { x = "Date"; y = "SMB" } +
        geomLine { x = "Date"; y = "HML" } +
        ggtitle("Factor Returns Over Time")

factorReturnsPlot.show()

### Histogram of Excess Returns:
This can help visualise the distribution of excess returns

In [98]:
val histogramData = mapOf("ExcessReturn" to y.toList())

val histogramPlot = ggplot(histogramData) +
        geomHistogram(bins = 30) { x = "ExcessReturn" } +
        ggtitle("Distribution of Excess Returns")

histogramPlot.show()